<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Deep-convolutional-GAN-(DCGAN)" data-toc-modified-id="Deep-convolutional-GAN-(DCGAN)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Deep convolutional GAN (DCGAN)</a></span></li></ul></div>

# Deep convolutional GAN (DCGAN)


• The network consisted of all convolutional layers. The pooling layers were
replaced by strided convolutions in the discriminator and transposed
convolutions in the generator.

• The fully connected classifying layers after the convolutions are removed.

• To help with the gradient flow, batch normalization is done after every
convolutional layer.

The basic idea of DCGANs is same as the vanilla GAN: we have a generator that
takes in noise of 100 dimensions; the noise is projected and reshaped, and then is
passed through convolutional layers. The following diagram shows the generator
architecture:

![](./i/DCGAN.png)

The discriminator network takes in the images (either generated by the generator
or from the real dataset), and the images undergo convolution followed by batch
normalization. At each convolution step the images get downsampled using strides.
The final output of the convolutional layer is flattened and feeds a one-neuron
classifier layer. In the following diagram, you can see the discriminator:

![](./i/DCGANdiscriminator.png)

The generator and the discriminator are combined together to form the DCGAN. The
training follows in the same manner as before; that is, we first train the discriminator
on a mini-batch, then freeze the discriminator and train the generator. The process is
repeated iteratively for a few thousand epochs. The authors found that we get more
stable results with the Adam optimizer and a learning rate of 0.002.

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np


Let us now build a DCGAN for generating handwritten digits. We first see the
code for the generator. The generator is built by adding the layers sequentially. The
first layer is a dense layer that takes the noise of 100 dimensions as an input. The
100-dimensional input is expanded to a flat vector of size 128 × 7 × 7. This is done so
that finally we get an output of size 28 × 28, the standard size of MNIST handwritten
digits. The vector is reshaped to a tensor of size 7 × 7 × 128. This vector is then
upsampled using TensorFlow Keras UpSampling2D layer. Please note that this layer
simply scales up the image by doubling rows and columns. The layer has no weights,
so it is computationally cheap.

In [2]:
class DCGAN():
    def __init__(self, rows, cols, channels, z = 10):
        # Input shape
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = z

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    '''The Upsampling2D layer will now double the rows and columns of the 7 × 7 × 128
        (rows × columns × channels) image, yielding an output of size 14 × 14 × 128. The
        upsampled image is passed to a convolutional layer. This convolutional layer learns
        to fill in the details in the upsampled image. The output of convolution is passed
        to batch normalization for better gradient flow. The batch normalized output then
        undergoes ReLU activation in all the intermediate layers. We repeat the structure,
        that is, upsampling | convolution | batch normalization | ReLU. In the following
        generator we have two such structures, the first with 128 filters, and the second
        with 64 filters in the convolution operation. The final output is obtained from
        a pure convolutional layer with 3 filters and tan hyperbolic activation, yielding
        an image of size 28 × 28 × 1:
    '''
    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))# a flat vector of size 128 × 7 × 7
        model.add(Reshape((7, 7, 128))) 
        model.add(UpSampling2D()) # upsampled using TensorFlow Keras UpSampling2D layer
        model.add(Conv2D(128, kernel_size=3, padding="same")) # now double the rows and columns of the 7 × 7 × 128 (rows × columns × channels)
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        print("********************************")
        print("Model summary of Generator ")
        print("********************************")
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)
    '''
        Now that we have a generator, let us see the code to build the discriminator. The
        discriminator is similar to a standard convolutional neural network but with one
        major change: instead of maxpooling we use convolutional layers with strides of
        2. We also add dropout layers to avoid overfitting, and batch normalization for
        better accuracy and fast convergence. The activation layer is leaky ReLU. In the
        following network we use three such convolutional layers, with filters of 32, 64, and
        128 respectively. The output of the third convolutional layer is flattened and fed
        to a dense layer with a single unit.
        The output of this unit classifies the image as fake or real:

    '''
    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        print("********************************")
        print("Model summary of Discriminator ")
        print("********************************")
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=256, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator is here
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator is here
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/dcgan_mnist_%d.png" % epoch)
        plt.close()


In [3]:
dcgan = DCGAN(28,28,1)
dcgan.train(epochs=5000, batch_size=256, save_interval=50)

********************************
Model summary of Discriminator 
********************************
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 64)          256       
________________________

61 [D loss: 0.874574, acc.: 47.27%] [G loss: 1.195105]
62 [D loss: 0.956581, acc.: 39.84%] [G loss: 1.134387]
63 [D loss: 0.794070, acc.: 53.32%] [G loss: 1.163544]
64 [D loss: 0.859538, acc.: 49.41%] [G loss: 1.142910]
65 [D loss: 0.984933, acc.: 42.97%] [G loss: 1.084019]
66 [D loss: 0.962688, acc.: 41.21%] [G loss: 0.973659]
67 [D loss: 1.010991, acc.: 39.26%] [G loss: 0.924999]
68 [D loss: 0.992673, acc.: 37.89%] [G loss: 0.930091]
69 [D loss: 1.006226, acc.: 38.87%] [G loss: 0.944239]
70 [D loss: 0.866521, acc.: 47.85%] [G loss: 1.001043]
71 [D loss: 0.889287, acc.: 45.12%] [G loss: 1.125386]
72 [D loss: 0.786522, acc.: 54.30%] [G loss: 1.020154]
73 [D loss: 0.613506, acc.: 65.23%] [G loss: 1.014942]
74 [D loss: 0.563310, acc.: 71.68%] [G loss: 0.908175]
75 [D loss: 0.610921, acc.: 69.53%] [G loss: 0.842030]
76 [D loss: 0.669392, acc.: 63.87%] [G loss: 0.891781]
77 [D loss: 0.885145, acc.: 47.85%] [G loss: 0.934034]
78 [D loss: 1.036613, acc.: 37.50%] [G loss: 0.936140]
79 [D loss

209 [D loss: 0.679617, acc.: 61.33%] [G loss: 1.116619]
210 [D loss: 0.648649, acc.: 62.50%] [G loss: 1.081104]
211 [D loss: 0.655283, acc.: 63.87%] [G loss: 1.122977]
212 [D loss: 0.600048, acc.: 69.14%] [G loss: 1.113318]
213 [D loss: 0.659555, acc.: 65.82%] [G loss: 1.123984]
214 [D loss: 0.655358, acc.: 62.89%] [G loss: 1.182457]
215 [D loss: 0.666095, acc.: 63.87%] [G loss: 1.141797]
216 [D loss: 0.679698, acc.: 60.35%] [G loss: 1.213860]
217 [D loss: 0.761978, acc.: 53.32%] [G loss: 1.267449]
218 [D loss: 0.841812, acc.: 47.85%] [G loss: 1.381550]
219 [D loss: 0.751236, acc.: 51.17%] [G loss: 1.392346]
220 [D loss: 0.659334, acc.: 64.45%] [G loss: 1.356638]
221 [D loss: 0.750606, acc.: 54.49%] [G loss: 1.172536]
222 [D loss: 0.679327, acc.: 59.77%] [G loss: 1.099972]
223 [D loss: 0.573410, acc.: 71.29%] [G loss: 0.971140]
224 [D loss: 0.554355, acc.: 70.90%] [G loss: 0.843486]
225 [D loss: 0.742215, acc.: 55.47%] [G loss: 0.855548]
226 [D loss: 0.645142, acc.: 63.87%] [G loss: 0.

357 [D loss: 0.649684, acc.: 62.70%] [G loss: 1.152503]
358 [D loss: 0.762505, acc.: 53.32%] [G loss: 1.056375]
359 [D loss: 0.877811, acc.: 42.77%] [G loss: 1.087864]
360 [D loss: 0.596721, acc.: 68.75%] [G loss: 1.122509]
361 [D loss: 0.692697, acc.: 58.59%] [G loss: 1.020348]
362 [D loss: 0.464019, acc.: 77.73%] [G loss: 1.156400]
363 [D loss: 0.556666, acc.: 70.70%] [G loss: 1.037416]
364 [D loss: 0.569140, acc.: 69.73%] [G loss: 1.099891]
365 [D loss: 0.703025, acc.: 58.01%] [G loss: 1.077180]
366 [D loss: 0.660217, acc.: 60.74%] [G loss: 1.088537]
367 [D loss: 0.506292, acc.: 75.20%] [G loss: 1.072358]
368 [D loss: 0.550488, acc.: 74.22%] [G loss: 1.136365]
369 [D loss: 0.471380, acc.: 80.08%] [G loss: 1.028644]
370 [D loss: 0.511360, acc.: 73.83%] [G loss: 1.143780]
371 [D loss: 0.711911, acc.: 58.20%] [G loss: 1.113876]
372 [D loss: 0.864110, acc.: 46.09%] [G loss: 1.131177]
373 [D loss: 0.901791, acc.: 41.41%] [G loss: 1.155340]
374 [D loss: 0.667643, acc.: 60.74%] [G loss: 1.

505 [D loss: 0.862830, acc.: 46.68%] [G loss: 0.875332]
506 [D loss: 0.469098, acc.: 77.54%] [G loss: 0.904764]
507 [D loss: 0.564971, acc.: 68.16%] [G loss: 0.819556]
508 [D loss: 0.308895, acc.: 90.82%] [G loss: 0.856953]
509 [D loss: 0.359690, acc.: 88.87%] [G loss: 0.881380]
510 [D loss: 0.478977, acc.: 78.12%] [G loss: 0.879670]
511 [D loss: 0.294219, acc.: 92.19%] [G loss: 0.919473]
512 [D loss: 0.355873, acc.: 86.52%] [G loss: 1.014998]
513 [D loss: 0.472848, acc.: 78.52%] [G loss: 1.033542]
514 [D loss: 0.355553, acc.: 88.09%] [G loss: 1.021989]
515 [D loss: 0.722020, acc.: 59.96%] [G loss: 0.980034]
516 [D loss: 0.381134, acc.: 85.74%] [G loss: 0.970529]
517 [D loss: 0.271045, acc.: 92.38%] [G loss: 1.053679]
518 [D loss: 0.463494, acc.: 78.12%] [G loss: 0.965244]
519 [D loss: 0.328878, acc.: 89.26%] [G loss: 1.059246]
520 [D loss: 0.429559, acc.: 79.88%] [G loss: 0.993883]
521 [D loss: 0.314475, acc.: 90.43%] [G loss: 1.021054]
522 [D loss: 0.673749, acc.: 61.91%] [G loss: 0.

...

4355 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.536604]
4356 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.425557]
4357 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.471916]
4358 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.504054]
4359 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.418077]
4360 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.398648]
4361 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.448958]
4362 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.391143]
4363 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.745831]
4364 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.465319]
4365 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.570935]
4366 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.446118]
4367 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.422460]
4368 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.543267]
4369 [D loss: 0.000003, acc.: 100.00%] [G loss: 2.640783]
4370 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.783180]
4371 [D loss: 0.000003, acc.: 100.00%] [G loss: 2.699525]
4372 [D loss: 

4497 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.688374]
4498 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.748796]
4499 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.707340]
4500 [D loss: 0.000003, acc.: 100.00%] [G loss: 2.688523]
4501 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.720250]
4502 [D loss: 0.000003, acc.: 100.00%] [G loss: 2.691595]
4503 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.634649]
4504 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.823607]
4505 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.615200]
4506 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.717161]
4507 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.585557]
4508 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.585097]
4509 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.533726]
4510 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.368117]
4511 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.736244]
4512 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.679909]
4513 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.402525]
4514 [D loss: 

4639 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.823542]
4640 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.714379]
4641 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.842826]
4642 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.870313]
4643 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.938156]
4644 [D loss: 0.000002, acc.: 100.00%] [G loss: 3.141891]
4645 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.006441]
4646 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.928171]
4647 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.044059]
4648 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.909787]
4649 [D loss: 0.000001, acc.: 100.00%] [G loss: 2.817404]
4650 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.992494]
4651 [D loss: 0.000002, acc.: 100.00%] [G loss: 2.945852]
4652 [D loss: 0.000002, acc.: 100.00%] [G loss: 3.139693]
4653 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.043724]
4654 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.026761]
4655 [D loss: 0.000002, acc.: 100.00%] [G loss: 3.007339]
4656 [D loss: 

4781 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.534722]
4782 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.372606]
4783 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.453758]
4784 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.313545]
4785 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.448701]
4786 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.418100]
4787 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.453068]
4788 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.561275]
4789 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.391356]
4790 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.534517]
4791 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.250126]
4792 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.617349]
4793 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.637711]
4794 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.413602]
4795 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.475002]
4796 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.457571]
4797 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.411088]
4798 [D loss: 

4923 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.621714]
4924 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.445790]
4925 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.563886]
4926 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.507718]
4927 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.572387]
4928 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.453650]
4929 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.629722]
4930 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.559608]
4931 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.540233]
4932 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.558001]
4933 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.459355]
4934 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.637266]
4935 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.455950]
4936 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.528785]
4937 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.495056]
4938 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.543715]
4939 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.491277]
4940 [D loss: 